In [ ]:
!git clone --depth 1 https://github.com/ACEsuit/mace.git

# Install MACE
!cd mace && pip install .

Cloning into 'mace'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 135 (delta 4), reused 43 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (135/135), 115.68 MiB | 29.46 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Processing /content/mace
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.7/387.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.0/449.0 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 125.

In [ ]:
#!pip install numpy==1.23.5
import os
import torch
from ase.calculators.emt import EMT
from mace.calculators import mace_mp
from ase.constraints import FixAtoms
from ase.io import read, write
from ase.mep import NEB
from ase.optimize import BFGS, QuasiNewton
import csv

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set base directory where your structures are stored
base_drive_path = "/content/drive/MyDrive/MLIP-NEB-inputs"  # Modify as needed

device = "cuda" if torch.cuda.is_available() else "cpu"

# Recursively walk through all subdirectories under base_drive_path
for root, dirs, files in os.walk(base_drive_path):
    # Check if current directory contains both "initial" and "final" subdirectories
    if 'initial' in dirs and 'final' in dirs:
        structure_path = root
        structure_name = os.path.basename(structure_path)

        # Define the RESULTS directory path and check if results already exist
        save_dir = os.path.join(structure_path, "MACE")
        csv_path = os.path.join(save_dir, "energies.csv")
        if os.path.exists(csv_path):
            print(f"Skipping {structure_name}: {csv_path} already exists.")
            continue

        # Define paths to initial and final POSCAR files
        initial_path = os.path.join(structure_path, "initial", "POSCAR")
        final_path = os.path.join(structure_path, "final", "POSCAR")
        if not os.path.exists(initial_path) or not os.path.exists(final_path):
            print(f"❌ Skipping {structure_name}: Missing POSCAR in initial or final directory")
            continue

        # Create the RESULTS directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        print(f"✅ Processing {structure_name}")
        try:
            initial = read(initial_path)
            final = read(final_path)
        except Exception as e:
            print(f"Error reading POSCAR for {structure_name}: {e}")
            continue

        # Fix atoms with tag > 1
        mask = [atom.tag > 1 for atom in initial]
        initial.set_constraint(FixAtoms(mask=mask))
        final.set_constraint(FixAtoms(mask=mask))

        # Assign calculator (MACE)
        calc = mace_mp(model="large", dispersion=False, default_dtype="float64", device=device)
        initial.calc = calc
        final.calc = calc

        # Relax initial structure
        try:
            qn = QuasiNewton(initial, trajectory=os.path.join(save_dir, "initial.traj"))
            qn.run(fmax=0.05, steps=1000)
        except RuntimeError as e:
            print(f"LineSearch failed during initial relaxation for {structure_name}: {e}")
            continue

        # Relax final structure
        try:
            qn = QuasiNewton(final, trajectory=os.path.join(save_dir, "final.traj"))
            qn.run(fmax=0.05, steps=1000)
        except RuntimeError as e:
            print(f"LineSearch failed during final relaxation for {structure_name}: {e}")
            continue

        # Save relaxed structures
        write(os.path.join(save_dir, "relaxed_initial.vasp"), initial, format="vasp")
        write(os.path.join(save_dir, "relaxed_final.vasp"), final, format="vasp")

        # Read relaxed structures (to ensure we're using the optimized geometry)
        try:
            initial = read(os.path.join(save_dir, "initial.traj"))
            final = read(os.path.join(save_dir, "final.traj"))
        except Exception as e:
            print(f"Error reading trajectories for {structure_name}: {e}")
            continue

        # Prepare images for NEB calculation
        images = [initial] + [initial.copy() for _ in range(5)] + [final]
        for image in images:
            image.calc = mace_mp(model="large", dispersion=False, default_dtype="float64", device=device)
            image.set_constraint(FixAtoms(mask=mask))

        try:
            neb = NEB(images, parallel=True, k=5, method='eb')
            neb.interpolate('idpp')
        except Exception as e:
            print(f"NEB interpolation failed for {structure_name}: {e}")
            continue

        try:
            qn = BFGS(neb, trajectory=os.path.join(save_dir, "neb.traj"))
            qn.run(fmax=0.05, steps=1000)
        except RuntimeError as e:
            print(f"LineSearch failed during NEB optimization for {structure_name}: {e}")
            continue

        # Store energies and save NEB images
        energies = []
        with open(csv_path, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Image Index", "Energy (eV)"])
            for i, image in enumerate(images):
                try:
                    energy = image.get_potential_energy()
                except Exception as e:
                    print(f"Error computing energy for image {i} of {structure_name}: {e}")
                    energy = None
                energies.append(energy)
                write(os.path.join(save_dir, f"neb_optimized_{i}.vasp"), image, format="vasp")
                writer.writerow([i, energy])

        if None in energies:
            print(f"Skipping migration barrier calculation for {structure_name} due to energy errors")
            continue

        migration_barrier = max(energies) - min(energies)
        print(f"{structure_name} - Migration Barrier: {migration_barrier:.6f} eV")
        with open(csv_path, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([])
            writer.writerow(["Migration Barrier", migration_barrier])

print("✅ All NEB Calculations complete!")







/usr/local/lib/python3.12/dist-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
Mounted at /content/drive
✅ Processing neb1
Cached MACE model to /root/.cache/mace/MACE_MPtrj_20229model
Using Materials Project MACE for MACECalculator with /root/.cache/mace/MACE_MPtrj_20229model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/usr/local/lib/python3.12/dist-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 14:32:57     -441.821083       1.4672
BFGSLineSearch:    1[  2] 14:32:58     -442.315709       0.7413
BFGSLineSearch:    2[  3] 14:32:59     -442.470233       0.3583
BFGSLineSearch:    3[  4] 14:33:00     -442.540549       0.2218
BFGSLineSearch:    4[  5] 14:33:01     -442.572857       0.1251
BFGSLineSearch:    5[  6] 14:33:01     -442.589648       0.0966
BFGSLineSearch:    6[  7] 14:33:01     -442.599260       0.0865
BFGSLineSearch:    7[  8] 14:33:01     -442.606241       0.0604
BFGSLineSearch:    8[  9] 14:33:02     -442.611413       0.0487
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 14:33:02     -441.821083       1.4672
BFGSLineSearch:    1[  2] 14:33:02     -442.315709       0.7413
BFGSLineSearch:    2[  3] 14:33:02     -442.470233       0.3583
BFGSLineSearch:    3[  4] 14:33:03     -442.540549       0.2218


/usr/local/lib/python3.12/dist-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


...

Using head Default out of ['Default']
Using Materials Project MACE for MACECalculator with /root/.cache/mace/MACE_MPtrj_20229model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/usr/local/lib/python3.12/dist-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Using Materials Project MACE for MACECalculator with /root/.cache/mace/MACE_MPtrj_20229model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/usr/local/lib/python3.12/dist-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
      Step     Time          Energy          fmax
BFGS:    0 15:16:59    -1198.209250        1.177235
BFGS:    1 15:17:02    -1198.249688        0.901645
BFGS:    2 15:17:06    -1198.324778        0.367373
BFGS:    3 15:17:11    -1198.333895        0.325148
BFGS:    4 15:17:15    -1198.353286        0.281973
BFGS:    5 15:17:18    -1198.361534        0.261799
BFGS:    6 15:17:23    -1198.370577        0.182616
BFGS:    7 15:17:27    -1198.376916        0.183642
BFGS:    8 15:17:31    -1198.382252        0.142792
BFGS:    9 15:17:35    -1198.386379        0.178523
BFGS:   10 15:17:39    -1198.390332        0.151614
BFGS:   11 15:17:43    -1198.393653        0.117645
BFGS:   12 15:17:47    -1198.396143        0.081598
BFGS:   13 15:17:51    -1198.397857        0.051077
BFGS:   14 15:17:55    -1198.399177        0.067533
BFGS:   15 15:17:59    -1198.400263        0.056526
BFGS:   16 15:18:03    -1198.401079        0.042039
neb2 - Migration Barrier: 0.